In [38]:
import re
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm

In [39]:
#https://stackoverflow.com/questions/2319019/using-regex-to-remove-comments-from-source-files
#https://github.com/HektorLin/HU-IRTG/blob/master/Partition%20Codes%20and%20Comments.ipynb
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group (capturing comments) is not None,
        # it means we have captured a non-quoted (real) comment string.
        if match.group(2) is not None:
            return "" # so we will return empty to remove the comment
        else: # otherwise, we will return the 1st group
            return match.group(1) # captured quoted-string
    return regex.sub(_replacer, string)

#following the example above, distinguish group(1): "" or'' and group(2,true comments): \*.*\ or //
#return when a true comment is located
def leave_only_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    comments = [x[1] for x in regex.findall(string)]
    comments = ' '.join(comments)
    return comments

In [40]:
files = os.listdir('sol_source')
df = pd.DataFrame(columns=['hash', 'name_from_SC', 'SC_full', 'SC_no_comments', 'comments'])

In [41]:
for file in tqdm(files):
    contract_hash = file.split('_')[0]
    name = file.split('_')[1].split('.')[0]
    with open(f'sol_source/{file}', 'r', encoding="utf8") as readf:
        contract = readf.read()
    df_temp = pd.DataFrame(data={'hash':contract_hash,
                                'name_from_SC': name,
                                'SC_full': contract,
                                'SC_no_comments': remove_comments(contract),
                                'comments': leave_only_comments(contract)}, index=[0])
    df = pd.concat([df, df_temp])
df.reset_index(drop=True, inplace=True)

100%|██████████████████████████████████████| 2187/2187 [00:41<00:00, 53.11it/s]


In [42]:
df_additional_info = pd.read_csv('../CSC_Dapp_scraping/dapps.csv')
df = df.merge(df_additional_info, how='left', left_on='hash', right_on='hash')

In [43]:
df.to_csv('dapps_with_SC_parsed.csv', index=False)